# JupyterHealth Exchange MCP Client with LLM Integration

This notebook demonstrates how to:
1. Connect to the JupyterHealth Exchange MCP server using the Python MCP SDK
2. Use the NRP-hosted Qwen3 LLM to process natural language queries
3. Enable the LLM to call MCP tools for health data retrieval
4. Display results in a user-friendly format

## Prerequisites

Before running this notebook:
- JHE Django server running on http://localhost:8000
- MCP server running on http://localhost:8001
- `.env` file configured with NRP API credentials
- OAuth authentication completed (token cached at ~/.jhe_mcp/token_cache.json)

## Cell 1: Setup and Imports

In [1]:
# FIRST: Fix Jupyter async event loop for MCP SDK compatibility
import nest_asyncio
nest_asyncio.apply()
print("✓ nest_asyncio applied - MCP SDK async compatibility enabled")

# Standard library
import os
import sys
import json
import asyncio
from typing import Any, Dict, List
from pathlib import Path

# Environment variables
from dotenv import load_dotenv

# MCP SDK
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client

# OpenAI client for NRP
from openai import OpenAI

# Utilities
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
import re

# Initialize console for rich output
console = Console()

# Load environment variables
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    console.print("[green]✓ Environment variables loaded from .env[/green]")
else:
    console.print("[yellow]⚠ .env file not found. Using defaults.[/yellow]")

# Display configuration
config_table = Table(title="Configuration")
config_table.add_column("Setting", style="cyan")
config_table.add_column("Value", style="green")

config_table.add_row("MCP Server URL", os.getenv("JHE_MCP_URL", "http://localhost:8001/sse"))
config_table.add_row("NRP Base URL", os.getenv("NRP_BASE_URL", "https://ellm.nrp-nautilus.io/v1"))
config_table.add_row("NRP Model", os.getenv("NRP_MODEL", "qwen3"))
config_table.add_row("NRP API Key", "*" * 20 if os.getenv("NRP_API_KEY") else "[red]NOT SET[/red]")

console.print(config_table)

✓ nest_asyncio applied - MCP SDK async compatibility enabled


✓ Environment variables loaded from .env

                   Configuration                    
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Setting        ┃ Value                           ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ MCP Server URL │ http://localhost:8001/sse       │
│ NRP Base URL   │ https://ellm.nrp-nautilus.io/v1 │
│ NRP Model      │ qwen3                           │
│ NRP API Key    │ ********************            │
└────────────────┴─────────────────────────────────┘

## Cell 2: MCP Client Connection

Connect to the JupyterHealth Exchange MCP server and discover available tools.

In [2]:
# MCP connection parameters
MCP_URL = os.getenv("JHE_MCP_URL", "http://localhost:8001/sse")

# Global variables for MCP session
mcp_session = None
mcp_tools = []
sse_context = None

async def connect_mcp():
    """Connect to the MCP server and initialize session"""
    global mcp_session, mcp_tools, sse_context
    
    console.print(f"[cyan]Connecting to MCP server at {MCP_URL}...[/cyan]")
    
    try:
        # Create SSE client connection with longer timeout
        sse_context = sse_client(url=MCP_URL, timeout=120.0)
        read, write = await sse_context.__aenter__()
        
        # Create MCP session
        mcp_session = ClientSession(read, write)
        await mcp_session.__aenter__()
        
        console.print("[yellow]Initializing MCP session...[/yellow]")
        
        # Initialize the session with timeout handling
        try:
            await asyncio.wait_for(mcp_session.initialize(), timeout=30.0)
        except asyncio.TimeoutError:
            console.print("[red]Timeout during initialization, but continuing...[/red]")
        
        console.print("[green]✓ Connected to MCP server[/green]")
        
        # List available tools
        try:
            tools_response = await asyncio.wait_for(mcp_session.list_tools(), timeout=10.0)
            mcp_tools = tools_response.tools
            
            console.print(f"[green]✓ Discovered {len(mcp_tools)} MCP tools[/green]\n")
            
            # Display tools in a table
            tools_table = Table(title="Available MCP Tools")
            tools_table.add_column("Tool Name", style="cyan", no_wrap=True)
            tools_table.add_column("Description", style="white")
            
            for tool in mcp_tools:
                # Truncate long descriptions
                desc = tool.description[:80] + "..." if len(tool.description) > 80 else tool.description
                tools_table.add_row(tool.name, desc)
            
            console.print(tools_table)
        except asyncio.TimeoutError:
            console.print("[yellow]⚠ Timeout listing tools, but connection established[/yellow]")
        
        return True
        
    except Exception as e:
        console.print(f"[red]✗ Failed to connect to MCP server: {e}[/red]")
        console.print(f"[dim]Error type: {type(e).__name__}[/dim]")
        console.print("[yellow]Make sure the MCP server is running on port 8001[/yellow]")
        import traceback
        console.print(f"[dim]{traceback.format_exc()}[/dim]")
        return False

# Connect to MCP server
connected = await connect_mcp()

Connecting to MCP server at http://localhost:8001/sse...

Initializing MCP session...

✓ Connected to MCP server

✓ Discovered 7 MCP tools

                                               Available MCP Tools                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tool Name                ┃ Description                                                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ get_study_count          │ Count total studies accessible to the authenticated user based on their role and... │
│ list_studies             │ List all studies accessible to the authenticated user with their IDs, names, and... │
│ get_patient_demographics │ Get patient demographics for a specific study. Returns patient IDs, ages, and em... │
│ get_study_metadata       │ Get metadata about a specific study including name, description, organization, p... │
│ get_patient_observations │ Get FHIR observations for a specific patient, including the FHIR/OMH JSONB data.... │
│ get_database_schema      │ Get comprehensive database schema information including table structures, column... │
│ execute_filtered_query   │ Execute an arbitrary SQL SELECT query with automatic permission filtering. Only ... │
└──────────────────────────┴─────────────────────────────────────────────────────────────────────────────────────┘

## Cell 3: NRP LLM Client Setup

Initialize the OpenAI client to connect to the NRP-hosted Qwen3 model.

In [3]:
# NRP LLM configuration
NRP_BASE_URL = os.getenv("NRP_BASE_URL", "https://ellm.nrp-nautilus.io/v1")
NRP_API_KEY = os.getenv("NRP_API_KEY")
NRP_MODEL = os.getenv("NRP_MODEL", "qwen3")

if not NRP_API_KEY:
    console.print("[red]ERROR: NRP_API_KEY not set in .env file[/red]")
    console.print("[yellow]Please add your NRP API key to the .env file[/yellow]")
else:
    # Initialize OpenAI client
    llm_client = OpenAI(
        base_url=NRP_BASE_URL,
        api_key=NRP_API_KEY
    )
    
    console.print("[green]✓ NRP LLM client initialized[/green]")
    
    # Test connectivity with a simple request
    try:
        response = llm_client.chat.completions.create(
            model=NRP_MODEL,
            messages=[{"role": "user", "content": "Hello! Reply with just 'OK' if you can hear me."}],
            max_tokens=10
        )
        
        console.print(f"[green]✓ NRP LLM connectivity test passed[/green]")
        response_content = response.choices[0].message.content
        if response_content:
            console.print(f"[dim]Response: {response_content}[/dim]")
        
    except Exception as e:
        console.print(f"[red]✗ Failed to connect to NRP LLM: {e}[/red]")
        console.print("[yellow]Check your NRP_API_KEY and NRP_BASE_URL in .env[/yellow]")

✓ NRP LLM client initialized

✓ NRP LLM connectivity test passed

## Cell 4: Simple MCP Tool Call

Demonstrate calling an MCP tool directly without LLM intervention.

In [4]:
async def call_mcp_tool(tool_name: str, arguments: Dict[str, Any] = None) -> Any:
    """Call an MCP tool and return the result"""
    if not mcp_session:
        console.print("[red]MCP session not initialized. Run Cell 2 first.[/red]")
        return None
    
    try:
        console.print(f"[cyan]Calling MCP tool: {tool_name}[/cyan]")
        if arguments:
            console.print(f"[dim]Arguments: {json.dumps(arguments, indent=2)}[/dim]")
        
        result = await mcp_session.call_tool(tool_name, arguments or {})
        
        console.print("[green]✓ Tool call successful[/green]\n")
        
        # Extract text content from result
        if hasattr(result, 'content') and len(result.content) > 0:
            content = result.content[0]
            if hasattr(content, 'text'):
                return content.text
        
        return str(result)
        
    except Exception as e:
        console.print(f"[red]✗ Tool call failed: {e}[/red]")
        return None

# Example: Get study count
console.print(Panel("[bold]Example: Get Study Count[/bold]", expand=False))
result = await call_mcp_tool("get_study_count")

if result:
    console.print(Panel(result, title="Result", border_style="green"))

╭──────────────────────────╮
│ Example: Get Study Count │
╰──────────────────────────╯

Calling MCP tool: get_study_count

✓ Tool call successful

╭──────────────────────────────────────────────────── Result ─────────────────────────────────────────────────────╮
│ You have access to 3 studies                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Cell 5: Explore Database Schema

Before running queries, let's understand the database structure using the `get_database_schema` tool.

In [5]:
# Cell 5: Explore Database Schema
# Before running queries, let's understand the database structure using the `get_database_schema` tool.

from rich import print as rprint
from rich.markdown import Markdown

rprint(Panel("[bold]Fetching Database Schema[/bold]", expand=False))

# Get the full database schema (returns formatted markdown text, not JSON)
schema_result = await call_mcp_tool("get_database_schema")

if schema_result:
    # Display the schema - render markdown nicely in Jupyter
    rprint("\n[bold cyan]Database Schema:[/bold cyan]\n")
    
    # Render markdown with Rich
    md = Markdown(schema_result)
    rprint(md)
    
    rprint("\n[green]✓ Schema exploration complete[/green]")
    rprint("[dim]Tip: The LLM can use this schema information with example SQL queries to construct accurate queries[/dim]")
else:
    rprint("[red]✗ Failed to retrieve database schema[/red]")


╭──────────────────────────╮
│ Fetching Database Schema │
╰──────────────────────────╯

Calling MCP tool: get_database_schema

✓ Tool call successful

Database Schema:

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                     JupyterHealth Exchange Database Schema                                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                    Core Tables                                                    

                                                    core_study                                                     

Clinical research studies                                                                                          

Columns:                                                                                                           

 • id (bigint): Study identifier (primary key)                                                                     
 • name (varchar): Study name                                                                                      
 • description (text): Study description                                                                           
 • organization_id (bigint): FK to core_organization                                                               
 • icon_url (text): Optional study icon URL                                                                        

Relationships:                                                                                                     

 • Belongs to one Organization (via organization_id)                                                               
 • Has many Patients (via core_studypatient M2M table)                                                             
 • Has many scope requests (via core_studyscoperequest)                                                            

                                                   core_patient                                                    

Patient records with demographics                                                                                  

Columns:                                                                                                           

 • id (bigint): Patient identifier                                                                                 
 • jhe_user_id (bigint): FK to core_jheuser                                                                        
 • identifier (varchar): External patient ID                                                                       
 • name_family (varchar): Family name                                                                              
 • name_given (varchar): Given name                                                                                
 • birth_date (date): Date of birth                                                                                
 • telecom_phone (varchar): Phone number                                                                           
 • last_updated (timestamptz): Last update timestamp                                                               

Relationships:                                                                                                     

 • Has one JheUser (via jhe_user_id)                                                                               
 • Belongs to many Organizations (via core_patientorganization M2M)                                                
 • Enrolled in many Studies (via core_studypatient M2M)                                                            
 • Has many Observations (subject_patient_id)                                                                      

                                                 core_observation                                                  

Health observations in FHIR format with OMH data                                                                   

Columns:                               

✓ Schema exploration complete

Tip: The LLM can use this schema information with example SQL queries to construct accurate queries

## Cell 6: MCP Tool to OpenAI Function Conversion

Convert MCP tool definitions to OpenAI function calling format.

In [6]:
def mcp_tool_to_openai_function(tool) -> Dict[str, Any]:
    """Convert an MCP tool to OpenAI function calling format"""
    return {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema
        }
    }

# Convert all MCP tools to OpenAI format
openai_tools = [mcp_tool_to_openai_function(tool) for tool in mcp_tools]

console.print(f"[green]✓ Converted {len(openai_tools)} MCP tools to OpenAI function format[/green]")

# Display one example
console.print("\n[bold]Example OpenAI Function Definition:[/bold]")
console.print(JSON(json.dumps(openai_tools[0], indent=2)))

✓ Converted 7 MCP tools to OpenAI function format

Example OpenAI Function Definition:

{
  "type": "function",
  "function": {
    "name": "get_study_count",
    "description": "Count total studies accessible to the authenticated user based on their role and organization 
permissions",
    "parameters": {
      "type": "object",
      "properties": {},
      "required": []
    }
  }
}

## Cell 7: LLM + MCP Integration Function

Create a function that handles the full LLM → MCP tool calling → response flow.

In [7]:
async def query_with_llm(user_query: str, max_iterations: int = 5) -> str:
    """
    Send a query to the LLM with access to MCP tools.
    The LLM can make function calls to retrieve health data.
    
    Args:
        user_query: Natural language query from the user
        max_iterations: Maximum number of tool calls to allow
        
    Returns:
        Final response from the LLM
    """
    console.print(Panel(f"[bold cyan]User Query:[/bold cyan] {user_query}", expand=False))
    
    # Initialize conversation with user query
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to a JupyterHealth Exchange database. "
                      "Use the available tools to answer questions about health data. "
                      "When you need database schema information, call get_database_schema first. "
                      "For complex queries, use execute_filtered_query with proper SQL."
        },
        {"role": "user", "content": user_query}
    ]
    
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        console.print(f"\n[dim]→ LLM iteration {iteration}/{max_iterations}[/dim]")
        
        # Call LLM with tools
        response = llm_client.chat.completions.create(
            model=NRP_MODEL,
            messages=messages,
            tools=openai_tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Check if LLM wants to call a tool
        if assistant_message.tool_calls:
            console.print(f"[yellow]LLM requested {len(assistant_message.tool_calls)} tool call(s)[/yellow]")
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                
                console.print(f"  [cyan]→ Calling: {tool_name}[/cyan]")
                
                # Call the MCP tool
                tool_result = await call_mcp_tool(tool_name, tool_args)
                
                # Add tool result to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result or "Error: Tool call failed"
                })
        else:
            # LLM has finished - return final response
            final_response = assistant_message.content
            console.print("\n[green]✓ LLM completed response[/green]")
            return final_response
    
    return "Maximum iterations reached. The query may be too complex."

console.print("[green]✓ LLM integration function ready[/green]")

✓ LLM integration function ready

## Cell 8: Simple Query Example

Test the integration with a simple query.

In [8]:
# Simple query
query = "How many studies are in the system?"

response = await query_with_llm(query)

console.print("\n" + "="*60)
console.print(Panel(response, title="[bold green]Final Response[/bold green]", border_style="green"))

╭─────────────────────────────────────────────────╮
│ User Query: How many studies are in the system? │
╰─────────────────────────────────────────────────╯

→ LLM iteration 1/5

LLM requested 1 tool call(s)

→ Calling: get_study_count

Calling MCP tool: get_study_count

✓ Tool call successful

→ LLM iteration 2/5

✓ LLM completed response

============================================================

╭──────────────────────────────────────────────── Final Response ─────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ The system contains **3 studies** that you have access to.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Cell 9: Complex Health Data Query

Query blood pressure data with filtering and aggregation using LLM.

In [10]:
# Cell 9: Complex Health Data Query
# Fetch schema first, then let LLM use it to construct the query

console.print(Panel("[bold]Complex Blood Pressure Query with Schema Context[/bold]", expand=False))

# Step 1: Get the database schema first
console.print("[cyan]Step 1: Fetching database schema...[/cyan]")
schema_result = await call_mcp_tool("get_database_schema")

if not schema_result:
    console.print("[red]✗ Failed to get schema[/red]")
else:
    console.print("[green]✓ Schema fetched successfully[/green]")
    
    # Step 2: Ask the LLM with schema context
    console.print("\n[cyan]Step 2: Asking LLM to query blood pressure data...[/cyan]")
    
    query = f"""I have access to a database with this schema:

{schema_result}

Now, using this schema, please:
Show me all blood pressure readings for patient 40001 on February 2, 2025.
Calculate the average systolic and diastolic pressure for that day.

Use the example SQL queries in the schema as a guide."""

    response = await query_with_llm(query, max_iterations=5)
    
    console.print("\n" + "="*60)
    console.print(Panel(response, title="[bold green]Final Response[/bold green]", border_style="green"))


╭──────────────────────────────────────────────────╮
│ Complex Blood Pressure Query with Schema Context │
╰──────────────────────────────────────────────────╯

Step 1: Fetching database schema...

Calling MCP tool: get_database_schema

✓ Tool call successful

✓ Schema fetched successfully

...

Calling MCP tool: execute_filtered_query

Arguments: {
  "sql_query": "SELECT AVG((value_attachment_data->'body'->'systolic_blood_pressure'->>'value')::numeric) AS 
avg_systolic, AVG((value_attachment_data->'body'->'diastolic_blood_pressure'->>'value')::numeric) AS avg_diastolic 
FROM core_observation WHERE subject_patient_id = 40001 AND codeable_concept_id IN (SELECT id FROM 
core_codeableconcept WHERE display ILIKE '%blood pressure%') AND 
(value_attachment_data->'body'->'effective_time_frame'->>'date_time')::timestamptz::date = '2025-02-02'"
}

✓ Tool call successful

============================================================

╭──────────────────────────────────────────────── Final Response ─────────────────────────────────────────────────╮
│ Maximum iterations reached. The query may be too complex.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Cell 10: Time-Series Aggregation Query

Demonstrate hourly aggregation with statistical functions.

In [ ]:
# Time-series aggregation query
query = """For patient 40001, show me hourly blood pressure statistics for February 2, 2025.
Include the average, minimum, maximum, and standard deviation for both systolic and diastolic pressure."""

response = await query_with_llm(query, max_iterations=10)

console.print("\n" + "="*60)
console.print(Panel(response, title="[bold green]Final Response[/bold green]", border_style="green"))

## Cell 11: Interactive Query Loop

Create an interactive interface for asking questions.

In [ ]:
async def interactive_session():
    """Run an interactive query session"""
    console.print(Panel(
        "[bold]JupyterHealth Exchange Interactive Query Session[/bold]\n\n"
        "Ask questions about health data in natural language.\n"
        "Type 'quit' or 'exit' to stop.",
        border_style="cyan"
    ))
    
    while True:
        # Get user input
        console.print("\n[bold cyan]Your question:[/bold cyan]", end=" ")
        user_input = input()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            console.print("[yellow]Goodbye![/yellow]")
            break
        
        if not user_input.strip():
            continue
        
        # Process query
        try:
            response = await query_with_llm(user_input)
            console.print("\n" + "="*60)
            console.print(Panel(response, title="[bold green]Response[/bold green]", border_style="green"))
        except Exception as e:
            console.print(f"[red]Error: {e}[/red]")

# Uncomment to run interactive session
await interactive_session()

console.print("[dim]Interactive session ready. Uncomment the last line to start.[/dim]")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ JupyterHealth Exchange Interactive Query Session                                                                │
│                                                                                                                 │
│ Ask questions about health data in natural language.                                                            │
│ Type 'quit' or 'exit' to stop.                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your question:

## Cell 12: Blood Pressure Visualization

Plot systolic and diastolic blood pressure data for February 2025.

In [ ]:
# Cell 12: LLM Creates Blood Pressure Visualization
# Ask the LLM to fetch data AND generate the visualization code

console.print(Panel("[bold]LLM-Powered Data Fetch & Visualization Creation[/bold]", expand=False))

# Ask the LLM to create a complete visualization
query = """For patient 40001, fetch all blood pressure data for February 2025 and create a visualization.

Please:
1. Query the database for systolic and diastolic blood pressure readings
2. Parse the results into a pandas DataFrame
3. Create a matplotlib line plot showing both systolic (red) and diastolic (blue) over time
4. Add reference lines at 120 mmHg (normal systolic) and 80 mmHg (normal diastolic)
5. Include a statistics table showing mean, median, std dev, min, max for both

Return the complete Python code needed to create this visualization."""

console.print("[cyan]Asking LLM to create visualization code...[/cyan]")
llm_response = await query_with_llm(query, max_iterations=15)

console.print("\n" + "="*60)
console.print(Panel(llm_response, title="[bold green]LLM Response[/bold green]", border_style="green"))

# Note: The LLM should return Python code that we can execute
# For safety and demonstration purposes, we'll execute it if it looks valid
console.print("\n[yellow]Note: In production, you would review and execute the LLM-generated code above.[/yellow]")
console.print("[dim]For this demo, the LLM has demonstrated its ability to:[/dim]")
console.print("[dim]  1. Understand database schema[/dim]")
console.print("[dim]  2. Construct SQL queries[/dim]")
console.print("[dim]  3. Parse results[/dim]")
console.print("[dim]  4. Generate visualization code[/dim]")


## Cell 13: Cleanup

Close connections and clean up resources.

In [ ]:
# Close MCP session
if mcp_session:
    try:
        await mcp_session.__aexit__(None, None, None)
        console.print("[green]✓ MCP session closed[/green]")
    except:
        pass

console.print("[green]✓ Cleanup complete[/green]")